In [1]:
import os
import numpy as np
import astropy.table as Table
import astropy.units as unit
import astropy.constants as cons
from astropy.io import fits
from astropy.table import Table
import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import astropy.coordinates as coord
from astroquery.gaia import Gaia
import warnings
from pyia import GaiaData
from matplotlib.patches import Ellipse
#import mpld3
from astropy.coordinates import (ICRS, GalacticLSR, CartesianDifferential, CartesianRepresentation)
import stilism_extinction as stil
from scipy import optimize
from scipy.interpolate import UnivariateSpline
import datetime
import time
import ratelimiter as RL
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.interpolate import UnivariateSpline
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
# #Below settings for presentation figs
# import seaborn.apionly as sns
# sns.set_context('paper')
# plt.rcParams['savefig.dpi']=800
# plt.rcParams['ytick.labelsize']='large'
# plt.rcParams['xtick.labelsize']='large'
# plt.rcParams['axes.labelsize']=20
# plt.rcParams['figure.figsize']=(12.8,9.6)

#plt.style.use('HCH_plotstyle')
#%matplotlib inline
#plt.rcParams['text.usetex'] = True
#plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
#from cycler import cycler
#plt.rcParams['axes.prop_cycle'] = cycler(color=['#000000','#E69F00','#56B4E9','#009E73','#F0E442','#0072B2','#D55E00','#CC79A7'])
# plt.rcParams["font.weight"] = "bold"
plt.rcParams['figure.figsize']=(2*3.35,2*.75*3.35)
#plt.rcParams['figure.dpi']=150
# plt.rcParams['font.size']=20
# plt.rcParams['font.family']='serif'
plt.rcParams['xtick.direction']='in'
plt.rcParams['ytick.direction']='in'
plt.rcParams['xtick.minor.visible']='True'
plt.rcParams['ytick.minor.visible']='True'

/home/jacobhhamer/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1003: UserWarning: Duplicate key in file "/home/jacobhhamer/.config/matplotlib/matplotlibrc", line #78
  (fname, cnt))
/home/jacobhhamer/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)


Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [6]:
def load_dr25_usps(gaia_ids=True, koi_score_cut=True):
    os.system('./DR25USPQuery.sh')
    dr25usps = pd.read_csv('DR25USPs.txt')
    dr25usps.rename(index=str,columns = {'ra':'ra_nasa','dec':'dec_nasa'}, inplace = True)
    dr25usps['KIC']=dr25usps['kepid'].astype(str)
    if koi_score_cut:
        dr25usps=dr25usps.loc[dr25usps.koi_score>0.68]
    dr25usps['pl_orbper'] = dr25usps.koi_period
    dr25usps['pl_orbsmax'] = dr25usps.koi_sma
    dr25usps['pl_hostname'] = 'KIC '+dr25usps.KIC
    dr25usps=dr25usps[['pl_hostname','pl_orbper','pl_orbsmax']]
    if gaia_ids:
        get_gaia_id_usps(dr25usps)
    return dr25usps

def load_confirmed_usps(gaia_ids=True):
    os.system('./ConfirmedUSPQuery.sh')
    confirmed_usps = pd.read_csv('confirmedUSPs.txt')
    confirmed_usps.rename(index=str,columns = {'ra':'ra_nasa','dec':'dec_nasa'}, inplace = True)
    confirmed_usps=confirmed_usps[['pl_hostname','pl_orbper','pl_orbsmax']]
    if gaia_ids:
        get_gaia_id_usps(confirmed_usps)
    return confirmed_usps

def load_non_kep_usps(gaia_ids=True):
    os.system('./NonKeplerUSPQuery.sh')
    nonkep_usps = pd.read_csv('nonkeplerUSPs.txt')
    nonkep_usps.rename(index=str,columns = {'ra':'ra_nasa','dec':'dec_nasa'}, inplace = True)
    nonkep_usps=nonkep_usps[['pl_hostname','pl_orbper','pl_orbsmax']]
    if gaia_ids:
        get_gaia_id_usps(nonkep_usps)
    return nonkep_usps

def load_so_usps(gaia_ids=True):
    usps = Table.read('../Data/asu.fit')
    usps = usps.to_pandas()
    usps['KIC'] = usps.KIC.str.decode('utf-8')
    sousps['pl_orbper'] = sousps.Porb
    sousps['pl_orbsmax'] = sousps.a_R_*sousps.R_*cons.R_sun.to('au')
    sousps['pl_hostname'] = 'KIC '+sousps.KIC
    sousps=sousps[['pl_hostname','pl_orbper','pl_orbsmax']]
    if gaia_ids:
        get_gaia_id_usps(sousps)
    return usps

def load_k2_usps(gaia_ids=True):
    k2usps=pd.read_csv('K2USP_10806/planet_table_machine.txt'\
                   , na_values=' ',index_col=False,header=None, skiprows=1\
                   , names=['Candidate','Campaign','Period','Period_errup','Period_errdn','t_0',\
                            't_0_errup','t_0_errdn','Duration','Duration_errup','Duration_errdn',\
                            'Depth','Depth_errup','Depth_errdn','RpRs','RpRs_errup','RpRs_errdn',\
                            'Rp','Rp_errup','Rp_errdn','Rstar','Rstar_errup','Rstar_errdn','aRstar',\
                            'aRstar_errup','aRstar_errdn','IncFlux','IncFlux_errup','IncFlux_errdn']\
                   , dtype={'Candidate':str,'Campaign':str,'Period':np.float64,'Period_errup':np.float64,\
                            'Period_errdn':np.float64,'t_0':np.float64,'t_0_errup':np.float64,\
                            't_0_errdn':np.float64,'Duration':np.float64,'Duration_errup':np.float64,\
                            'Duration_errdn':np.float64,'Depth':np.float64,'Depth_errup':np.float64,\
                            'Depth_errdn':np.float64,'RpRs':np.float64,'RpRs_errup':np.float64,\
                            'RpRs_errdn':np.float64,'Rp':np.float64,'Rp_errup':np.float64,\
                            'Rp_errdn':np.float64,'Rstar':np.float64,'Rstar_errup':np.float64,\
                            'Rstar_errdn':np.float64,'aRstar':np.float64,'aRstar_errup':np.float64,\
                            'aRstar_errdn':np.float64,'IncFlux':np.float64,'IncFlux_errup':np.float64,\
                            'IncFlux_errdn':np.float64})
    k2usps=k2usps.loc[k2usps.Period+3*k2usps.Period_errup<1]
    k2usps['pl_orbper'] = k2usps.Period
    k2usps['pl_orbsmax'] = k2usps.aRstar*k2usps.Rstar*cons.R_sun.to('au')
    k2usps['pl_hostname'] = 'EPIC'+k2usps.Candidate[:-2]
    k2usps=k2usps[['pl_hostname','pl_orbper','pl_orbsmax']]
    if gaia_ids:
        get_gaia_id_usps(k2usps)
    return k2usps

def get_gaia_id_usps(data):
    Simbad.add_votable_fields('ids')
    ids=[]
    #ratel=RL.RateLimiter(4, period=1)
    for j in range(0,(len(data)//100)+1):
        if j<(len(data)//100):
            #print(j)
            for i, sys in data.iloc[j*100:(j+1)*100].iterrows():
                #print(j,i)
                #with ratel:
                s=Simbad.query_object(sys.pl_hostname)
                if s is not None:
                    s=s.to_pandas()
                    s['IDS']=s['IDS'].str.decode('utf-8')
                    gaiaid=[idx for idx in s['IDS'][0].split('|') if 'Gaia DR2' in idx]
                    if len(gaiaid)>0:
                        ids.append(gaiaid[0])
                    else:
                        ids.append(np.nan)
                else:
                    ids.append(np.nan)
            time.sleep(20)
        else:
            for i, sys in data.iloc[j*100:j*100+len(data)%100].iterrows():
                #print(j,i)
                #with ratel:
                s=Simbad.query_object(sys.pl_hostname)
                if s is not None:
                    s=s.to_pandas()
                    s['IDS']=s['IDS'].str.decode('utf-8')
                    gaiaid=[idx for idx in s['IDS'][0].split('|') if 'Gaia DR2' in idx]
                    if len(gaiaid)>0:
                        ids.append(gaiaid[0])
                    else:
                        ids.append(np.nan)
                else:
                    ids.append(np.nan)
    data['gaia_id']=ids

def construct_best_id(data):
    """Adds field 'best_id' of HIP/TYC IDs to the array, querying objects in SIMBAD for all alternative IDs.
    Args: 
        data - array
    Notes: 
        Gaia DR1 only uses HIP/TYC IDS. This looks for those IDS. Gaia DR2 has multiple best_neighbor tables, but
        this only does these two for now.
        First attempts all names, including names like HIP ##### A, but then strips the trailing A or B.
    """
    data['best_id'] = np.ones(len(data))*np.nan
    for i, system in data.iterrows():
        simquery = Simbad.query_objectids(system.pl_hostname)
        if simquery is None:
            name = system.pl_hostname[:-2] + system.pl_hostname[-2:].replace(' A','').replace(' B','')
            simquery = Simbad.query_objectids(name, cache=False)
        if simquery is None:
            continue
        tycid = [objid for objid in simquery.to_pandas()['ID'].values.astype(str) if 'TYC' in objid]
        hipid = [objid for objid in simquery.to_pandas()['ID'].values.astype(str) if 'HIP' in objid]
        if len(hipid)>0:
            data.loc[i, 'best_id'] = hipid[0]
        elif len(tycid)>0:
            data.loc[i, 'best_id'] = tycid[0]
        else:
            continue
    warnings.resetwarnings()
    
    
def gaia_xmatch_usps(data, unique = True):
    """Crossmatches the hot Jupiter data with Gaia DR2, first matching objects with HIP/TYC IDs and then performing positional searches.
    
    If unique, returns one Gaia object per hot Jupiter host.
    """
    
    gaia_bool = ~data.gaia_id.isnull()
    gaia_preided = data.loc[gaia_bool]
    gaiaids = str(tuple(gaia_preided.gaia_id.values))
    query = "SELECT gaia.designation, gaia.ra, gaia.ra_error, gaia.dec, gaia.dec_error, gaia.parallax, gaia.parallax_error, gaia.parallax_over_error, gaia.pmra, gaia.pmra_error, gaia.pmdec, gaia.pmdec_error, gaia.ra_dec_corr, gaia.ra_parallax_corr, gaia.ra_pmra_corr, gaia.ra_pmdec_corr, gaia.dec_parallax_corr, gaia.dec_pmra_corr, gaia.dec_pmdec_corr, gaia.parallax_pmra_corr, gaia.parallax_pmdec_corr, gaia.pmra_pmdec_corr, gaia.astrometric_n_obs_al, gaia.astrometric_n_obs_ac, gaia.astrometric_n_good_obs_al, gaia.astrometric_n_bad_obs_al, gaia.astrometric_gof_al, gaia.astrometric_chi2_al, gaia.astrometric_excess_noise, gaia.astrometric_excess_noise_sig, gaia.visibility_periods_used, gaia.mean_varpi_factor_al,  gaia.rv_nb_transits, gaia.phot_g_mean_mag, gaia.bp_rp, gaia.bp_g, gaia.g_rp, gaia.radial_velocity, gaia.radial_velocity_error, gaia.phot_variable_flag, gaia.l, gaia.b FROM gaiadr2.gaia_source AS gaia WHERE gaia.designation IN " + gaiaids
    gaiajob = Gaia.launch_job(query)
    gaiaided = gaiajob.get_data()
    gaiaided = gaiaided.to_pandas()
    gaiaided['designation'] = gaiaided['designation'].str.decode('utf-8')
    gaia_prematched = pd.merge(data, gaiaided, how='left',left_on='gaia_id', right_on='designation')
    print('Have simbad DR2 ids:',len(gaia_prematched.loc[~gaia_prematched.designation.isnull()]))
    
    srcids = gaia_prematched.loc[~gaia_prematched.designation.isnull()].designation.apply(lambda row: row.replace('Gaia DR2 ','')).astype('long')
    gaia_prematched.loc[~gaia_prematched.designation.isnull(),'source_id'] = gaia_prematched.loc[~gaia_prematched.designation.isnull()].designation.apply(lambda row: row.replace('Gaia DR2 ','')).astype('long')
    print(len(gaia_prematched.loc[~gaia_prematched.source_id.isnull()]))
    source_ids = str(tuple(srcids.values))
    designations = str(tuple(gaia_prematched.loc[~gaia_prematched.designation.isnull()].designation.values))
    bjonesquery = 'SELECT gaia.designation, bjones.r_est as "distance", bjones.r_lo, bjones.r_hi FROM external.gaiadr2_geometric_distance as bjones INNER JOIN gaiadr2.gaia_source as gaia ON bjones.source_id = gaia.source_id WHERE gaia.designation IN '+designations
    job = Gaia.launch_job(bjonesquery)
    jobdata = job.get_data()
    hjbjones = jobdata.to_pandas()
    hjbjones['designation']=hjbjones['designation'].str.decode('utf-8')
    #hjbjones=pd.read_csv('../Data/HotJupBailerJones.csv')#,dtype={'source_id':'long'})
    print('bjones dists',len(hjbjones.loc[~hjbjones.distance.isnull()]))
    gaia_prematched = pd.merge(gaia_prematched, hjbjones, how='left', left_on='designation', right_on='designation')
    print('after bjones merge on designation',len(gaia_prematched.loc[~gaia_prematched.distance.isnull()]))
    return gaia_prematched

def prepare_dr25_usp_sample(nongiants = True, delta_M_G0_cut = 1, bailer_jones = True,preloaded = False, kinematics = True, plx_prec_cut=True, clean_astrometry=True):
    if preloaded:
        matchedusps = pd.read_csv('../Data/gaia_dr25_usp_xmatch.csv')
    else:
        dr25=load_dr25_usps()
        nonkep=load_non_kep_usps()
        k2=load_k2_usps()
        usps=pd.concat([dr25,nonkep,k2])
        matchedusps = gaia_xmatch_usps(usps,unique = True)
        matchedusps.to_csv('../Data/gaia_dr25_usp_xmatch.csv')
    if plx_prec_cut:
        matchedusps=matchedusps.loc[matchedusps.parallax_over_error>10]
    if bailer_jones==False:
        matchedusps['distance'] = 1000 / matchedusps['parallax']
    if clean_astrometry:
        matchedusps=astrometrically_clean_USPs(matchedusps)
    extinction_correct_Gaia(matchedusps)
    if nongiants:
        matchedusps = keep_nongiants(matchedusps, delta_M_G0_cut = delta_M_G0_cut)
    if kinematics:
        matchedusps.to_csv('../Data/matched_dr25_usps.csv')
        gd = GaiaData('../Data/matched_dr25_usps.csv')
        c = gd.skycoord
        gal = c.galacticlsr
        gal.set_representation_cls('cartesian')
        matchedusps['gal_U'] = gal.v_x
        matchedusps['gal_V'] = gal.v_y
        matchedusps['gal_W'] = gal.v_z
        matchedusps['gal_u'] = gal.x
        matchedusps['gal_v'] = gal.y
        matchedusps['gal_w'] = gal.z
        matchedusps.to_csv('../Data/matched_dr25_usp_kinematics.csv')
    #sweetcat=pd.read_table('../Data/SWEET-Cat.txt',header=None, na_values='~')
    #sweetcat.rename(columns={0:"Name",1:"HD_number",2:"RA",3:"Dec",4:"Vmag",5:"e_Vmag",6:"plx",7:"e_plx",8:"plx_source",9:"Teff ",10:"e_Teff ",11:"logg",12:"e_logg",13:"LC_logg",14:"e_LC_logg",15:"Vt",16:"e_Vt",17:"[Fe/H]",18:"e_[Fe/H]",19:"Mass",20:"e_Mass",21:"Reference",22:"Homogeneity_flag",23:"Last_Update",24:"Comments"},inplace=True)
    #for col in sweetcat.columns:
    #    if (col=="Name" or col=='Reference' or col=='Last_Update' or col=='Comments' or col=='plx_source' or 'HD_number'):
    #        continue
    #    elif col=='Homogeneity_flag':
    #        sweetcat[col]=sweetcat[col].astype(int)
    #    else:
    #        sweetcat[col]=sweetcat[col].astype(float)
    #sweetcat=sweetcat.loc[sweetcat.Homogeneity_flag==1]
    #matchedusps=pd.merge(matchedusps,sweetcat, left_on='pl_hostname', right_on='Name',how='left')
    return matchedusps

def prepare_so_usp_sample(nongiants = True, delta_M_G0_cut = 1, bailer_jones = True,preloaded = False, kinematics = True, plx_prec_cut=True, clean_astrometry=True):
    if preloaded:
        matchedusps = pd.read_csv('../Data/gaia_so_usp_xmatch.csv')
    else:
        so=load_so_usps()
        nonkep=load_non_kep_usps()
        k2=load_k2_usps()
        usps=pd.concat([so,nonkep,k2])
        matchedusps = gaia_xmatch_usps(usps,unique = True)
        matchedusps.to_csv('../Data/gaia_so_usp_xmatch.csv')
    if plx_prec_cut:
        matchedusps=matchedusps.loc[matchedusps.parallax_over_error>10]
    if bailer_jones==False:
        matchedusps['distance'] = 1000 / matchedusps['parallax']
    if clean_astrometry:
        matchedusps=astrometrically_clean_USPs(matchedusps)
    extinction_correct_Gaia(matchedusps)
    if nongiants:
        matchedusps = keep_nongiants(matchedusps, delta_M_G0_cut = delta_M_G0_cut)
    if kinematics:
        matchedusps.to_csv('../Data/matched_so_usps.csv')
        gd = GaiaData('../Data/matched_so_usps.csv')
        c = gd.skycoord
        gal = c.galacticlsr
        gal.set_representation_cls('cartesian')
        matchedusps['gal_U'] = gal.v_x
        matchedusps['gal_V'] = gal.v_y
        matchedusps['gal_W'] = gal.v_z
        matchedusps['gal_u'] = gal.x
        matchedusps['gal_v'] = gal.y
        matchedusps['gal_w'] = gal.z
        matchedusps.to_csv('../Data/matched_so_usp_kinematics.csv')
    #sweetcat=pd.read_table('../Data/SWEET-Cat.txt',header=None, na_values='~')
    #sweetcat.rename(columns={0:"Name",1:"HD_number",2:"RA",3:"Dec",4:"Vmag",5:"e_Vmag",6:"plx",7:"e_plx",8:"plx_source",9:"Teff ",10:"e_Teff ",11:"logg",12:"e_logg",13:"LC_logg",14:"e_LC_logg",15:"Vt",16:"e_Vt",17:"[Fe/H]",18:"e_[Fe/H]",19:"Mass",20:"e_Mass",21:"Reference",22:"Homogeneity_flag",23:"Last_Update",24:"Comments"},inplace=True)
    #for col in sweetcat.columns:
    #    if (col=="Name" or col=='Reference' or col=='Last_Update' or col=='Comments' or col=='plx_source' or 'HD_number'):
    #        continue
    #    elif col=='Homogeneity_flag':
    #        sweetcat[col]=sweetcat[col].astype(int)
    #    else:
    #        sweetcat[col]=sweetcat[col].astype(float)
    #sweetcat=sweetcat.loc[sweetcat.Homogeneity_flag==1]
    #matchedusps=pd.merge(matchedusps,sweetcat, left_on='pl_hostname', right_on='Name',how='left')
    return matchedusps

def gaia_xmatch(data, unique = True):
    """Crossmatches the hot Jupiter data with Gaia DR2, first matching objects with HIP/TYC IDs and then performing positional searches.
    
    If unique, returns one Gaia object per hot Jupiter host.
    """
    
    gaia_bool = ~data.gaia_id.isnull()
    gaia_preided = data.loc[gaia_bool]
    gaiaids = str(tuple(gaia_preided.gaia_id.values))
    query = "SELECT gaia.designation, gaia.ra, gaia.ra_error, gaia.dec, gaia.dec_error, gaia.parallax, gaia.parallax_error, gaia.parallax_over_error, gaia.pmra, gaia.pmra_error, gaia.pmdec, gaia.pmdec_error, gaia.ra_dec_corr, gaia.ra_parallax_corr, gaia.ra_pmra_corr, gaia.ra_pmdec_corr, gaia.dec_parallax_corr, gaia.dec_pmra_corr, gaia.dec_pmdec_corr, gaia.parallax_pmra_corr, gaia.parallax_pmdec_corr, gaia.pmra_pmdec_corr, gaia.astrometric_n_obs_al, gaia.astrometric_n_obs_ac, gaia.astrometric_n_good_obs_al, gaia.astrometric_n_bad_obs_al, gaia.astrometric_gof_al, gaia.astrometric_chi2_al, gaia.astrometric_excess_noise, gaia.astrometric_excess_noise_sig, gaia.visibility_periods_used, gaia.mean_varpi_factor_al,  gaia.rv_nb_transits, gaia.phot_g_mean_mag, gaia.bp_rp, gaia.bp_g, gaia.g_rp, gaia.radial_velocity, gaia.radial_velocity_error, gaia.phot_variable_flag, gaia.l, gaia.b FROM gaiadr2.gaia_source AS gaia WHERE gaia.designation IN " + gaiaids
    gaiajob = Gaia.launch_job(query)
    gaiaided = gaiajob.get_data()
    gaiaided = gaiaided.to_pandas()
    gaiaided['designation'] = gaiaided['designation'].str.decode('utf-8')
    gaia_prematched = pd.merge(data, gaiaided, how='left',left_on='gaia_id', right_on='designation')
    print('Have simbad DR2 ids:',len(gaia_prematched.loc[~gaia_prematched.designation.isnull()]))
    
    construct_best_id(data)
    idable = data.gaia_id.isnull()
    idablehjs = data.loc[idable]
    idablehjs = idablehjs.loc[~idablehjs.best_id.isnull()]
    hiphjs = idablehjs.loc[~idablehjs.hip_name.isnull()]
    tychjs = idablehjs.loc[idablehjs.hip_name.isnull()]
    hipids = str(tuple(hiphjs.best_id.values)).replace('HIP ','').replace(' A','').replace(' B','')
    tycids = str(tuple(tychjs.best_id.values)).replace('TYC ','').replace(' A','').replace(' B','')
    if ((len(tychjs.best_id.values)>2)&(len(hiphjs.best_id.values)>2)):
        #query = "SELECT gaia.designation, gaia.source_id, gaia.ra AS ra_gaia, gaia.ra_error AS ra_error_gaia, gaia.dec AS dec_gaia, gaia.dec_error AS dec_error_gaia, gaia.parallax, gaia.parallax_error, gaia.parallax_over_error, gaia.pmra, gaia.pmra_error, gaia.pmdec, gaia.pmdec_error, gaia.radial_velocity, gaia.radial_velocity_error, gaia.phot_g_mean_mag, gaia.bp_rp, xmatch.original_ext_source_id FROM gaiadr2.gaia_source AS gaia INNER JOIN gaiadr2.hipparcos2_best_neighbour AS xmatch ON gaia.source_id = xmatch.source_id WHERE xmatch.original_ext_source_id IN " + hipids
        query = "SELECT gaia.designation, gaia.ra, gaia.ra_error, gaia.dec, gaia.dec_error, gaia.parallax, gaia.parallax_error, gaia.parallax_over_error, gaia.pmra, gaia.pmra_error, gaia.pmdec, gaia.pmdec_error, gaia.ra_dec_corr, gaia.ra_parallax_corr, gaia.ra_pmra_corr, gaia.ra_pmdec_corr, gaia.dec_parallax_corr, gaia.dec_pmra_corr, gaia.dec_pmdec_corr, gaia.parallax_pmra_corr, gaia.parallax_pmdec_corr, gaia.pmra_pmdec_corr, gaia.astrometric_n_obs_al, gaia.astrometric_n_obs_ac, gaia.astrometric_n_good_obs_al, gaia.astrometric_n_bad_obs_al, gaia.astrometric_gof_al, gaia.astrometric_chi2_al, gaia.astrometric_excess_noise, gaia.astrometric_excess_noise_sig, gaia.visibility_periods_used, gaia.mean_varpi_factor_al,  gaia.rv_nb_transits, gaia.phot_g_mean_mag, gaia.bp_rp, gaia.bp_g, gaia.g_rp, gaia.radial_velocity, gaia.radial_velocity_error, gaia.phot_variable_flag, gaia.l, gaia.b, xmatch.original_ext_source_id FROM gaiadr2.gaia_source AS gaia INNER JOIN gaiadr2.hipparcos2_best_neighbour AS xmatch ON gaia.source_id = xmatch.source_id WHERE xmatch.original_ext_source_id IN " + hipids
        hipjob = Gaia.launch_job(query)
        #query = "SELECT gaia.designation, gaia.source_id, gaia.ra AS ra_gaia, gaia.ra_error AS ra_error_gaia, gaia.dec AS dec_gaia, gaia.dec_error AS dec_error_gaia, gaia.parallax, gaia.parallax_error, gaia.parallax_over_error, gaia.pmra, gaia.pmra_error, gaia.pmdec, gaia.pmdec_error, gaia.radial_velocity, gaia.radial_velocity_error, gaia.phot_g_mean_mag, gaia.bp_rp, xmatch.original_ext_source_id FROM gaiadr2.gaia_source AS gaia INNER JOIN gaiadr2.tycho2_best_neighbour AS xmatch ON gaia.source_id = xmatch.source_id WHERE xmatch.original_ext_source_id IN " + tycids
        query = "SELECT gaia.designation, gaia.ra, gaia.ra_error, gaia.dec, gaia.dec_error, gaia.parallax, gaia.parallax_error, gaia.parallax_over_error, gaia.pmra, gaia.pmra_error, gaia.pmdec, gaia.pmdec_error, gaia.ra_dec_corr, gaia.ra_parallax_corr, gaia.ra_pmra_corr, gaia.ra_pmdec_corr, gaia.dec_parallax_corr, gaia.dec_pmra_corr, gaia.dec_pmdec_corr, gaia.parallax_pmra_corr, gaia.parallax_pmdec_corr, gaia.pmra_pmdec_corr, gaia.astrometric_n_obs_al, gaia.astrometric_n_obs_ac, gaia.astrometric_n_good_obs_al, gaia.astrometric_n_bad_obs_al, gaia.astrometric_gof_al, gaia.astrometric_chi2_al, gaia.astrometric_excess_noise, gaia.astrometric_excess_noise_sig, gaia.visibility_periods_used, gaia.mean_varpi_factor_al,  gaia.rv_nb_transits, gaia.phot_g_mean_mag, gaia.bp_rp, gaia.bp_g, gaia.g_rp, gaia.radial_velocity, gaia.radial_velocity_error, gaia.phot_variable_flag, gaia.l, gaia.b, xmatch.original_ext_source_id FROM gaiadr2.gaia_source AS gaia INNER JOIN gaiadr2.tycho2_best_neighbour AS xmatch ON gaia.source_id = xmatch.source_id WHERE xmatch.original_ext_source_id IN " + tycids
        tycjob = Gaia.launch_job(query)
        hipgaia = hipjob.get_data()
        hipgaiapandas = hipgaia.to_pandas()
        hipgaiapandas['original_ext_source_id'] = hipgaiapandas['original_ext_source_id'].values.astype(int).astype(str)
        hipgaiapandas['original_ext_source_id'] = 'HIP ' + hipgaiapandas['original_ext_source_id']
        tycgaia = tycjob.get_data()
        tycgaiapandas = tycgaia.to_pandas()
        tycgaiapandas['original_ext_source_id'] = tycgaiapandas['original_ext_source_id'].astype(str)
        tycgaiapandas['original_ext_source_id'] = 'TYC ' + tycgaiapandas['original_ext_source_id']
        idablegaia = pd.concat([hipgaiapandas,tycgaiapandas])
        idablegaia['designation'] = idablegaia['designation'].str.decode('utf-8')
        idable_xmatch = pd.merge(data,idablegaia,how = 'left',left_on = 'best_id',right_on = 'original_ext_source_id')
        print('From best neighbor:',len(idable_xmatch.loc[~idable_xmatch.designation.isnull()]))
        print('Check the merges have same len',len(gaia_prematched),len(idable_xmatch))
    else:
        idable_xmatch = data
        idable_xmatch['designation']=np.ones(len(data))*np.nan
    
    nomatch1 = gaia_prematched.designation.isnull()
    nomatch2 = idable_xmatch.designation.isnull()
    in_neither = nomatch1 & nomatch2
    match1 = gaia_prematched.loc[~gaia_prematched.designation.isnull()]
    print('Check simbad set have designations', len(match1.loc[~match1.designation.isnull()]))
    match2 = idable_xmatch.loc[~idable_xmatch.designation.isnull()]
    print('Check best neighbor set have designations', len(match2.loc[~match2.designation.isnull()]))
    nogaiamatch = data.loc[in_neither]
    print(len(nogaiamatch))
    nogaiamatch = nogaiamatch.drop_duplicates(subset='pl_hostname')
    get_simbad_coordinates_pms_mags(nogaiamatch)

    frames = []
    for i, system in nogaiamatch.iterrows():
        if ~np.isnan(system.Simbad_RA):
            ra = system.Simbad_RA
        else:
            ra = system.ra_nasa
        if ~np.isnan(system.Simbad_DEC):
            dec = system.Simbad_DEC
        else:
            dec = system.dec_nasa
        print(ra,dec, system.pl_hostname)
        #query = "SELECT DISTANCE(EPOCH_PROP_POS(ra, dec, parallax, pmra, pmdec, radial_velocity, 2015.5, 2000),POINT('ICRS'," + str(ra) + "," + str(dec) + ")) AS angular_distance_deg, gaia.designation, gaia.source_id, gaia.ra AS ra_gaia, gaia.ra_error AS ra_error_gaia, gaia.dec AS dec_gaia, gaia.dec_error AS dec_error_gaia, gaia.parallax, gaia.parallax_error, gaia.parallax_over_error, gaia.pmra, gaia.pmra_error, gaia.pmdec, gaia.pmdec_error, gaia.radial_velocity, gaia.radial_velocity_error, gaia.phot_g_mean_mag, gaia.bp_rp FROM gaiadr2.gaia_source AS gaia WHERE 1 = CONTAINS(POINT('ICRS',ra, dec),CIRCLE('ICRS'," + str(ra) + "," + str(dec) + ",0.005555556))"
        query = "SELECT DISTANCE(EPOCH_PROP_POS(ra, dec, parallax, pmra, pmdec, radial_velocity, ref_epoch, 2000),POINT('ICRS'," + str(ra) + "," + str(dec) + "))*3600. AS angular_distance_as, gaia.designation, gaia.ra, gaia.ra_error, gaia.dec, gaia.dec_error, gaia.parallax, gaia.parallax_error, gaia.parallax_over_error, gaia.pmra, gaia.pmra_error, gaia.pmdec, gaia.pmdec_error, gaia.ra_dec_corr, gaia.ra_parallax_corr, gaia.ra_pmra_corr, gaia.ra_pmdec_corr, gaia.dec_parallax_corr, gaia.dec_pmra_corr, gaia.dec_pmdec_corr, gaia.parallax_pmra_corr, gaia.parallax_pmdec_corr, gaia.pmra_pmdec_corr, gaia.astrometric_n_obs_al, gaia.astrometric_n_obs_ac, gaia.astrometric_n_good_obs_al, gaia.astrometric_n_bad_obs_al, gaia.astrometric_gof_al, gaia.astrometric_chi2_al, gaia.astrometric_excess_noise, gaia.astrometric_excess_noise_sig, gaia.visibility_periods_used, gaia.mean_varpi_factor_al,  gaia.rv_nb_transits, gaia.phot_g_mean_mag, gaia.bp_rp, gaia.bp_g, gaia.g_rp, gaia.radial_velocity, gaia.radial_velocity_error, gaia.phot_variable_flag, gaia.l, gaia.b FROM gaiadr2.gaia_source AS gaia WHERE 1 = CONTAINS(POINT('ICRS',ra, dec),CIRCLE('ICRS'," + str(ra) + "," + str(dec) + ",0.002777777777777778))"
        job = Gaia.launch_job(query)
        jobdata = job.get_data()
        jobdatapandas = jobdata.to_pandas()
        jobdatapandas['original_object'] = [system.pl_hostname for i in range(len(jobdata))]
        frames.append(jobdatapandas)
    positionalgaia = pd.concat(frames)
    positional_xmatch = pd.merge(nogaiamatch, positionalgaia, how = 'outer', left_on = 'pl_hostname', right_on = 'original_object')
    print('Found in positional',len(positional_xmatch))
    positional_xmatch = convert_gaia_photometry_xmatch(positional_xmatch)
    print('After cutting positional based on mags',len(positional_xmatch))
    fullxmatch = pd.concat([match1, match2, positional_xmatch])
    print('match1, match2, full should equal',len(match1.loc[~match1.designation.isnull()]),len(match2.loc[~match2.designation.isnull()]),len(fullxmatch.loc[~fullxmatch.designation.isnull()]))
    #ided = ~fullxmatch.designation_x.isnull()
    #positioned = ~fullxmatch.designation_y.isnull()    
    #for col in [col for col in fullxmatch.columns if '_x' in col]:
    #    col = col.replace('_x','')
    #    fullxmatch.loc[ided, col] = fullxmatch.loc[ided, col + '_x']
    #    fullxmatch.loc[positioned, col] = fullxmatch.loc[positioned, col + '_y']
    #    fullxmatch.drop(col + '_x', axis = 1, inplace = True)
    #    fullxmatch.drop(col + '_y', axis = 1, inplace = True)
    fullxmatch.drop_duplicates(subset = 'designation',keep = 'first',inplace = True)
    print('after drop duplicates',len(fullxmatch.loc[~fullxmatch.designation.isnull()]))
    fullxmatch.loc[fullxmatch.angular_distance_as.isnull(),'angular_distance_as'] = 0.
    print('after ang dist fix',len(fullxmatch.loc[~fullxmatch.designation.isnull()]))
    #fullxmatch['designation'] = fullxmatch['designation'].str.decode('utf-8')
    #print('after designation decode',len(fullxmatch.loc[~fullxmatch.designation.isnull()]))
    #if unique:
    #    fullxmatch = fullxmatch.loc[fullxmatch.designation != 'Gaia DR2 6780546169633474944']
    #    fullxmatch = fullxmatch.loc[fullxmatch.designation != 'Gaia DR2 2080056067370451456']
    #print('after unique',len(fullxmatch.loc[~fullxmatch.designation.isnull()]))
    srcids = fullxmatch.loc[~fullxmatch.designation.isnull()].designation.apply(lambda row: row.replace('Gaia DR2 ','')).astype('long')
    fullxmatch.loc[~fullxmatch.designation.isnull(),'source_id'] = fullxmatch.loc[~fullxmatch.designation.isnull()].designation.apply(lambda row: row.replace('Gaia DR2 ','')).astype('long')
    print(len(fullxmatch.loc[~fullxmatch.source_id.isnull()]))
    source_ids = str(tuple(srcids.values))
    designations = str(tuple(fullxmatch.loc[~fullxmatch.designation.isnull()].designation.values))
    bjonesquery = 'SELECT gaia.designation, bjones.r_est as "distance", bjones.r_lo, bjones.r_hi FROM external.gaiadr2_geometric_distance as bjones INNER JOIN gaiadr2.gaia_source as gaia ON bjones.source_id = gaia.source_id WHERE gaia.designation IN '+designations
    job = Gaia.launch_job(bjonesquery)
    jobdata = job.get_data()
    hjbjones = jobdata.to_pandas()
    hjbjones['designation']=hjbjones['designation'].str.decode('utf-8')
    #hjbjones=pd.read_csv('../Data/HotJupBailerJones.csv')#,dtype={'source_id':'long'})
    print('bjones dists',len(hjbjones.loc[~hjbjones.distance.isnull()]))
    fullxmatch = pd.merge(fullxmatch, hjbjones, how='left', left_on='designation', right_on='designation')
    print('after bjones merge on designation',len(fullxmatch.loc[~fullxmatch.distance.isnull()]))
    return fullxmatch


def convert_gaia_photometry_xmatch(data):
    """Converts the Gaia photometry to Johnson system to allow for verification of crossmatch by apparent magnitude.
    
    See https://gea.esac.esa.int/archive/documentation/GDR2/Data_processing/chap_cu5pho/sec_cu5pho_calibr/ssec_cu5pho_PhotTransf.html
    """
    data['gaia_V'] = data['phot_g_mean_mag'] + 0.1732*data['bp_rp']**2 + 0.006860*data['bp_rp'] + 0.01760
    data = data.loc[np.abs(data['gaia_V']-data['Simbad_FLUX_V'])<1]
    return data

def abs_G_mag(data):
    data['distance_pc'] = 1000./data['parallax']
    data['G_mag_abs'] = data['phot_g_mean_mag'] + 5-5*np.log10(data['distance_pc'])
    

def compute_delta_M_G0(data):
    """Calulate deviation from main-sequence M_G - bp_rp color polynomial relation derived from the Pleiades Gaia DR2 data. 
    Used to remove giants.
    
    Args:
        data - array of data with extinction and reddening corrected Gaia photometry.
    """
    tcks=([-0.1533484 , -0.1533484 , -0.1533484 , -0.1533484 ,  0.63099294,
         0.96258741,  1.30771737,  1.71082406,  2.1444674 ,  2.4671927 ,
         2.5106812 ,  2.57087331,  2.61055761,  2.64182287,  2.65866666,
         2.66738897,  2.69233232,  2.70922952,  2.77087217,  2.82155137,
         2.84336095,  2.8537417 ,  2.89164453,  2.919838  ,  2.95142656,
         2.96471505,  2.98843198,  3.01530652,  3.04317003,  3.05686288,
         3.07316022,  3.10154062,  3.12891012,  3.16318994,  3.21736532,
         3.27362256,  3.3727818 ,  3.68525129,  3.68525129,  3.68525129,
         3.68525129],[-0.1745295 ,  2.3230733 ,  2.54252414,  5.54631992,  6.59988118,
         7.76297267,  8.59023705,  9.17519988,  9.65855025,  9.77533754,
         9.90392849, 10.05051825, 10.13809934, 10.19273253, 10.21350097,
        10.24293815, 10.38722777, 10.6498557 , 10.67855244, 10.68741252,
        10.71899853, 10.92565851, 10.97186376, 11.00763527, 11.08921082,
        11.27865818, 11.34078682, 11.34813415, 11.35886986, 11.42840745,
        11.55213609, 11.6855607 , 11.71061756, 12.12229352, 12.26016258,
        12.22448534, 12.8920296 ,  0.        ,  0.        ,  0.        ,
         0.        ],3)
    spl2=UnivariateSpline._from_tck(tcks)
    #a=np.array([ -0.0491425 ,   0.81243621,  -5.52471938,  19.74213029,-39.20921495,  42.06379052, -22.26781161,   8.91746038, 0.7786397 ])
    data['delta_M_G0'] = np.ones(len(data))*np.nan
    spline_val=spl2(data.bp_rp0)
    #poly = np.polyval(a,data.bp_rp0)
    #data['delta_M_G0'] = poly+5-5*np.log10(135.81630262048824)-data['M_G0'] #Average distance to Pleiades
    #data['delta_M_G0']=poly-data['M_G0']  # as poly is currently already in abs mag
    data['delta_M_G0']=spline_val-data['M_G0']  # as poly is currently already in abs mag
    return data


def plot_pleiades_main_sequence_poly(ax, minbprp = 0.2, maxbprp = 2.0):
    """Plot the main sequence polynomial derived from the Pleiades."""
    a=np.array([ -0.0491425 ,   0.81243621,  -5.52471938,  19.74213029,-39.20921495,  42.06379052, -22.26781161,   8.91746038, 0.7786397 ])
    bprps = np.linspace(minbprp, maxbprp, 100)
    polys = np.polyval(a,bprps)
    ax.plot(bprps, polys, zorder = 10, lw=2, c='k',ls='-.',alpha=0.75, label='Pleiades Main Sequence')
    return

def keep_nongiants(data, delta_M_V_cut = 1., delta_M_G0_cut=1., gaia=True):
    if gaia:       
        data = compute_delta_M_G0(data)
        nongiants = (data.loc[data.delta_M_G0<delta_M_G0_cut])
    else:
        data = compute_delta_M_V(data)
        nongiants = (data.loc[data.delta_M_V<delta_M_V_cut])
    return nongiants

def giants_nongiants_HR(data, delta_M_V_cut = 1, delta_M_G0_cut=1, gaia=True):
    f,ax=plt.subplots(1)
    if gaia:
        data = compute_delta_M_G0(data)
        nongiants = (data.loc[data.delta_M_G0<delta_M_G0_cut])
        giants = (data.loc[data.delta_M_G0>delta_M_G0_cut])
        plot_pleiades_main_sequence_poly(ax)
        if len(data)>1000:
            ax.scatter(nongiants.bp_rp0, nongiants.M_G0, color = 'b',s=5,alpha=0.02)
            ax.scatter(giants.bp_rp0, giants.M_G0, color = 'r',s=5,alpha=0.02)
        else:
            ax.scatter(nongiants.bp_rp0, nongiants.M_G0, color = 'b',s=10,alpha=0.6)
            ax.scatter(giants.bp_rp0, giants.M_G0, color = 'r',s=10,alpha=0.6)    
        ax.set_xlabel('$(G_{BP}-G_{RP})_0$')
        ax.set_ylabel('$M_{G0}$')
    else:
        data = compute_delta_M_V(data)
        nongiants = (data.loc[data.delta_M_V<delta_M_V_cut])
        giants = (data.loc[data.delta_M_V>delta_M_V_cut])
        #plot_main_sequence_poly(.44, 1.2)
        if len(data)>1000:
            ax.scatter(nongiants.B_V0, nongiants.V0, color = 'b',s=5,alpha=0.02)
            ax.scatter(giants.B_V0, giants.V0, color = 'r',s=5,alpha=0.02)
        else:
            ax.scatter(nongiants.B_V0, nongiants.V0, color = 'b',s=10,alpha=0.6)
            ax.scatter(giants.B_V0, giants.V0, color = 'r',s=10,alpha=0.6)
        ax.set_xlabel('$(B-V)_0$')
        ax.set_ylabel('$M_V0$')
    ax.invert_yaxis()
    return f,ax
                 
                 
def sp_type_vel_scatter(ax, hotjups, control, bprpbounds, comp1, comp2, both_ellipse=True):
    if both_ellipse:
        ax.grid(alpha = 0.15) 
        hjbprp = hotjups.bp_rp0
        hotjup_sptype = hotjups.loc[np.logical_and(hjbprp>bprpbounds[0],hjbprp<bprpbounds[1])]
        xhj = hotjup_sptype['gal_' + comp1].dropna().values
        yhj = hotjup_sptype['gal_' + comp2].dropna().values
        xstdhj = xhj.std()
        ystdhj = yhj.std()
        ellshj = [Ellipse((xhj.mean(),yhj.mean()), (1 + i)*xstdhj, (1 + i)*ystdhj) for i in range(3)]
        for i, e in enumerate(ellshj):
            e.set_clip_box(ax.bbox)
            e.set_alpha(0.75/(i+1))
            e.set_fill(False)
            e.set_ec('b')
            e.set_linewidth(4)
            ax.add_artist(e)

        controlbprp = control.bp_rp0
        control_sptype = control.loc[np.logical_and(controlbprp>bprpbounds[0], controlbprp<bprpbounds[1])]
        xcontrol = control_sptype['gal_' + comp1].dropna().values
        ycontrol = control_sptype['gal_' + comp2].dropna().values
        xstdcontrol = xcontrol.std()
        ystdcontrol = ycontrol.std()
        ellscontrol = [Ellipse((xcontrol.mean(),ycontrol.mean()), (1 + i)*xstdcontrol, (1 + i)*ystdcontrol) for i in range(3)]
        for e in ellscontrol:
            e.set_clip_box(ax.bbox)
            e.set_alpha(0.15)
            e.set_facecolor('k')
            ax.add_artist(e)
    else:
        ax.grid(alpha = 0.15)
        hjbprp = hotjups.bp_rp0
        hotjup_sptype = hotjups.loc[np.logical_and(hjbprp>bprpbounds[0],hjbprp<bprpbounds[1])]
        controlbprp = control.bp_rp0
        control_sptype = control.loc[np.logical_and(controlbprp>bprpbounds[0], controlbprp<bprpbounds[1])]
        x = control_sptype['gal_' + comp1].dropna().values
        y = control_sptype['gal_' + comp2].dropna().values
        xstd = x.std()
        ystd = y.std()
        ells = [Ellipse((x.mean(),y.mean()), (1 + i)*xstd, (1 + i)*ystd) for i in range(3)]
        for e in ells:
            e.set_clip_box(ax.bbox)
            e.set_alpha(0.15)
            e.set_facecolor('k')
            ax.add_artist(e)
        ax.scatter(hotjup_sptype['gal_' + comp1], hotjup_sptype['gal_' + comp2], s = 10)
    ax.set_ylabel('${}$ [km/s]'.format(comp2), labelpad = -15, fontsize = 24)
    ax.set_yticks([-50,-25,0,25,50])
    ax.set_xticks([-50,-25,0,25,50])
    ax.xaxis.set_minor_locator(AutoMinorLocator(4))
    ax.yaxis.set_minor_locator(AutoMinorLocator(4))
    ax.tick_params(axis='both', which='major', labelsize=20)
    ax.tick_params(top=True, left=True, right=True, bottom=True, which='both')
    ax.annotate(s='N={}'.format(len(hotjup_sptype)), xy=(.70,.84),xycoords='axes fraction', fontsize=26)
        
def all_sp_vel_scatter(hotjups, control, both_ellipse=True):
    f, ax = plt.subplots(3,3, figsize = (21,21),sharex=True)
    bounds = ((.438,.742),(.742,1.002),(1.002,2.02))
    sptypes = ('F', 'G', 'K')
    for i, bound in enumerate(bounds):
        sp_type_vel_scatter(ax[i][0], hotjups, control, bound, 'U', 'V', both_ellipse)
        sp_type_vel_scatter(ax[i][1], hotjups, control, bound, 'U', 'W', both_ellipse)
        sp_type_vel_scatter(ax[i][2], hotjups, control, bound, 'V', 'W', both_ellipse)
        for j in range(3):
            if i==2:
                ax[i][j].set_xlabel('${}$ [km/s]'.format('UUV'[j]), labelpad = 0, fontsize = 24)
            ax[i][j].annotate(s = '{} Stars'.format(sptypes[i]), xy = (.13,.84), xycoords = 'axes fraction', fontsize = 26)
            ax[i][j].set_ylim(-65,65)
            ax[i][j].set_xlim(-65,65)
    f.tight_layout()
    f.subplots_adjust(hspace=0.01,wspace=.20)
    return f, ax


def total_deviation(data):
    total_deviation = np.sum(np.sqrt((data.gal_U-data.gal_U.mean())**2 + \
                                 (data.gal_V-data.gal_V.mean())**2 + \
                                 (data.gal_W-data.gal_W.mean())**2))
    return total_deviation

def match_zdistribution(hotjups, control):
    hj_z=hotjups['gal_w']
    control_z=control['gal_w']
    control=control.loc[(control_z>hj_z.min())&(control_z<hj_z.max())]
    control_z=control_z.loc[(control_z>hj_z.min())&(control_z<hj_z.max())]
    hjhist,hjedges=np.histogram(hj_z,bins=12,density=True)
    ghist,gedges=np.histogram(control_z,bins=12,density=True)
    maxdisc=np.argmax(hjhist/ghist)
    ideal_total=int(len(control.loc[(control.gal_w>hjedges[maxdisc])&(control.gal_w<hjedges[maxdisc+1])])/(hjhist[maxdisc]/hjhist.sum()))
    indices=[]
    for i,frac in enumerate(hjhist/hjhist.sum()):
        try:
            bin_indices=control.loc[(control.gal_w>hjedges[i])&(control.gal_w<hjedges[i+1])].sample(int(ideal_total*frac)).index.values
        except ValueError:
            bin_indices=control.loc[(control.gal_w>hjedges[i])&(control.gal_w<hjedges[i+1])].index.values
        indices.append(bin_indices)
    indices=np.concatenate(indices)
    return control.loc[indices]


def mc_control_deviations(hjs, control, niters = 1000, metallicity=False, z_distribution=True, delta_M_G0_cut=1.):
    mciters = niters
    hjnongiants = keep_nongiants(hjs,delta_M_G0_cut=delta_M_G0_cut)
    nongiants = keep_nongiants(control,delta_M_G0_cut=delta_M_G0_cut)
    total_deviations = np.ones(mciters)*np.nan
    samps = []
    for i in range(mciters):
        if metallicity:
            control_sample=match_metallicity(hjnongiants, nongiants)
        if z_distribution:
            control_sample=match_zdistribution(hjnongiants, nongiants)
        else: 
            control_sample=nongiants
        bprpcontrol = control_sample.bp_rp0
        if i%(mciters/10) == 0:
            print('i = ',i, datetime.datetime.now())
        control_sample_inds = []
        for j, hj in hjnongiants.iterrows():
            bprprange = 0.025
            bprp = hj.bp_rp0
            bounds = (bprp-bprprange,bprp + bprprange)
            selection=control_sample.loc[np.logical_and(bprpcontrol>bounds[0], bprpcontrol<bounds[1])]
            while len(selection)==0:
                bprprange+=.005
                bounds = (bprp-bprprange,bprp + bprprange)
                selection = control_sample.loc[np.logical_and(bprpcontrol>bounds[0], bprpcontrol<bounds[1])]
            control_sample_inds.append(selection.sample(n=1).index.values)
        control_sample_inds = np.concatenate(control_sample_inds)
        control_sample = nongiants.loc[control_sample_inds]
        total_deviationi = total_deviation(control_sample)
        total_deviations[i] = total_deviationi/len(control_sample[~control_sample.gal_U.isnull()])
        samps.append(control_sample)
    #(hjnongiants.Bmag-hjnongiants.Vmag).hist()
    #(control_sample.Bmag-control_sample.Vmag).hist()
    plt.scatter(hjnongiants.bp_rp0, hjnongiants.M_G0, label = 'Gaia HJs')
    plt.scatter(control_sample.bp_rp0, control_sample.M_G0, label = 'Final Control Sample')
    plt.gca().invert_yaxis()
    return total_deviations, samps

def HJ_vs_control_dispersion_hist(hjs, control, mciters = 15000, metallicity=False, z_distribution=True, delta_M_G0_cut=1.):
    hjdevs = total_deviation(hjs)/len(hjs[~hjs.gal_U.isnull()])
    control_devs, samps = mc_control_deviations(hjs = hjs, control = control, niters = mciters, metallicity=metallicity, z_distribution=z_distribution, delta_M_G0_cut=delta_M_G0_cut)
    #fig = plt.figure()
    #plt.hist(control_devs, bins = 100)
    #maxy = plt.gca().get_ylim()[1]
    #plt.vlines(hjdevs, ymin = 0, ymax = maxy, label = 'HJ Total Dev', color = 'orange')
    #plt.vlines(x = np.percentile(control_devs,50), ymin = 0, ymax = maxy, label = 'Control Median', color = 'k')
    #plt.vlines(x = np.percentile(control_devs,[16,84]), ymin = 0, ymax = maxy, label = '1 $\\sigma$', color = 'k', linestyle = '--')
    #plt.vlines(x = np.percentile(control_devs,[2,98]), ymin = 0, ymax = maxy, label = '2 $\\sigma$', color = 'k', linestyle = '--',alpha=0.5)
    #plt.xlabel('Velocity Dispersion')
    #plt.legend()
    #plt.title('{} Iterations'.format(mciters))
    #plt.ylabel('Number of Monte Carlo Samples')
    #plt.annotate(s = "$HJs:{:.3f}\%$".format(len(control_devs[control_devs<hjdevs])*100/len(control_devs)), xy = (0.1,.5), xycoords = 'axes fraction', fontsize = 14)
    return hjdevs, control_devs, samps
def extinction_correct_Johnson(data):
    data['ext_B_V'] = np.ones(len(data))*np.nan
    for i, sys in data.iterrows():
        data.loc[i, 'ext_B_V'] = stil.ext_calc(sys.l, sys.b, sys.distance, 100) 
    data['ext_V'] = 3.1*data['ext_B_V']
    data['V_mag_abs']=data['Vmag']+5-5*np.log10(data['distance'])
    data['V0'] = data['V_mag_abs']-data['ext_V']
    data['B_V0'] = data['B_V']-data['ext_B_V']
    
def extinction_correct_Gaia(data):
    data['ext_B_V'] = np.ones(len(data))*np.nan
    for i, sys in data.iterrows():
        data.loc[i, 'ext_B_V'] = stil.ext_calc(sys.l, sys.b, sys.distance, 100) 
    data['ext_bp_rp'] = (3.374-2.035)*data['ext_B_V'] # Casagrande & VandenBerg 2018 Table 2 
    data['ext_G'] = 2.740*data['ext_B_V'] # Casagrande & VandenBerg 2018 Table 2 
    data['bp_rp0'] = data['bp_rp']-data['ext_bp_rp']
    data['G0'] = data['phot_g_mean_mag']-data['ext_G']
    data['M_G0'] = data['phot_g_mean_mag']+5-5*np.log10(data['distance'])


def astrometrically_clean_hot_jups(hotjups):
    cut1 = hotjups.parallax_over_error>10
    cut2 = hotjups.astrometric_gof_al<3
    cut3 = np.logical_and(hotjups.mean_varpi_factor_al>-0.23,hotjups.mean_varpi_factor_al<0.36)
    cut4 = hotjups.visibility_periods_used>8
    #NEXT ONLY WORKS BECAUSE EXP(G-19.5) > 1 FOR ALL HOT JUP HOSTS, not best implementation
    cut5 = np.sqrt(hotjups.astrometric_chi2_al/(hotjups.astrometric_n_good_obs_al-5)) < 1.2*np.exp(-0.2*(hotjups.phot_g_mean_mag-19.5))
    #allcuts = (((cut1 & cut2) & cut3) & cut4) & cut5
    allcuts = ((cut1 & cut3) & cut4) & cut5
    print('Before cleaning: ',len(hotjups))
    print('Limiting to parallax_over_error>10: ',len(hotjups.loc[cut1]))
    print('Limiting to astrometric_gof_al<3: ',len(hotjups.loc[cut2]))
    print('Limiting to -0.23<mean_varpi_factor_al<0.36: ',len(hotjups.loc[cut3]))
    print('Limiting to visibility_periods_used>8: ',len(hotjups.loc[cut4]))
    print('Limiting to Lindegren cut C.1: ',len(hotjups.loc[cut5]))
    hotjups=hotjups.loc[allcuts]
    print('All cuts except gof_al:', len(hotjups))
    return hotjups

def prepare_astrometrically_clean_gaia(preloaded=False,nongiants = True, delta_M_G0_cut = 1, fgk_only = False, bailer_jones = True, kinematics = True, metallicities=False):
    if preloaded:
        gaia=pd.read_csv('../Data/GAIADR2_Clean_Astrometry_extincted.csv')
    else:
        gaia=pd.read_csv('../Data/Gaia_rough_main_sequence.csv',dtype={'source_id':'long'})
        gaia['source_id'] = gaia.designation.apply(lambda row: row.replace('Gaia DR2 ','')).astype('long')
        if bailer_jones==False:
            gaia['distance'] = 1000/gaia['parallax']

        extinction_correct_Gaia(gaia)
        
        #gaia['BTmag'] = gaia['G0'] + 0.02441 + 0.4899 * gaia['bp_rp0'] + 0.9740 * gaia['bp_rp0']**2 + 0.2496 * gaia['bp_rp0']**3
        #gaia['VTmag'] = gaia['G0'] + 0.01842 + 0.06629 * gaia['bp_rp0'] + 0.2346 * gaia['bp_rp0']**2 + 0.02157 * gaia['bp_rp0']**3
        #gaia['Vmag'] = gaia['VTmag']-0.09*(gaia['BTmag']-gaia['VTmag'])
        #gaia['B_V0'] = 0.85*(gaia['BTmag']-gaia['VTmag'])

        #gaia['V0'] = gaia['Vmag']+5-5*np.log10(gaia['distance'])
        
        #gaia['BTmag'] = gaia['phot_g_mean_mag'] + 0.02441 + 0.4899 * gaia['bp_rp'] + 0.9740 * gaia['bp_rp']**2 + 0.2496 * gaia['bp_rp']**3
        #gaia['VTmag'] = gaia['phot_g_mean_mag'] + 0.01842 + 0.06629 * gaia['bp_rp'] + 0.2346 * gaia['bp_rp']**2 + 0.02157 * gaia['bp_rp']**3
        #gaia['Vmag'] = gaia['VTmag']-0.09*(gaia['BTmag']-gaia['VTmag'])
        #gaia['B_V'] = 0.85*(gaia['BTmag']-gaia['VTmag'])

        #gaia['V0'] = gaia['Vmag']+5-5*np.log10(gaia['distance'])
        #extinction_correct_Johnson(gaia)
        gaia.to_csv('../Data/GAIADR2_Clean_Astrometry_extincted.csv')
    if fgk_only:
        print('Before FGK cut: ',len(gaia))
        gaia = gaia.loc[np.logical_and(gaia.B_V0>0.44,gaia.B_V0<1.2)]
        print('After FGK cut: ',len(gaia))
    if nongiants:
        print('Before MS cut: ',len(gaia))
        gaia = keep_nongiants(gaia, delta_M_G0_cut = delta_M_G0_cut,gaia=True)
        print('After MS cut: ', len(gaia))
    if kinematics:
        gaia.to_csv('../Data/gaia_nongiants.csv')
        gd = GaiaData('../Data/gaia_nongiants.csv')
        c = gd.skycoord
        gal = c.galacticlsr
        gal.set_representation_cls('cartesian')
        gaia['gal_U'] = gal.v_x
        gaia['gal_V'] = gal.v_y
        gaia['gal_W'] = gal.v_z
        gaia['gal_u'] = gal.x
        gaia['gal_v'] = gal.y
        gaia['gal_w'] = gal.z
        gaia.to_csv('../Data/gaia_nongiants_kinematics.csv')
    return gaia


In [ ]:
dr25usps=prepare_dr25_usp_sample(nongiants=False, preloaded=False, kinematics=True,plx_prec_cut=False,clean_astrometry=True)

In [ ]:
sousps=prepare_so_usp_sample(nongiants=False, preloaded=False, kinematics=True,plx_prec_cut=False,clean_astrometry=True)

In [ ]:
usps_ms=keep_nongiants(usps, delta_M_G0_cut=1.)

In [ ]:
len(usps_ms)

In [ ]:
plt.scatter(usps_ms.bp_rp0, usps_ms.M_G0)
plt.ylim(10,0)

In [ ]:
f,ax=all_sp_vel_scatter(usps_ms,match_zdistribution(usps_ms,gaiams))
f.savefig('PaperPlots/USPVelocityEllipses.pdf',bbox_inches='tight')
f.savefig('PaperPlots/USPVelocityEllipses.png',bbox_inches='tight')

In [ ]:
uspdev, uspcontroldevs, uspsamps=HJ_vs_control_dispersion_hist(usps_ms, gaiams, mciters=10000)

In [ ]:
font={'size':16}
plt.rc('font',**font)
fig = plt.figure(figsize=(7.1,7.1*.5))
plt.hist(uspcontroldevs, bins = 100)
#maxy = plt.gca().get_ylim()[1]
plt.axvline(uspdev, label = 'USP Hosts',lw=5, color='orange')
#plt.vlines(hjdev, ymin = 0, ymax = maxy, label = 'HJ Total Dev', color = 'orange',lw=10)
plt.axvline(x = np.percentile(uspcontroldevs,50), label = 'Median', lw=3,color = 'k')
plt.axvline(x = np.percentile(uspcontroldevs,16,), lw=3,color = 'k', linestyle = '--',label='$1 \sigma$')
plt.axvline(x = np.percentile(uspcontroldevs,84), lw=3,color = 'k', linestyle = '--')
plt.axvline(x = np.percentile(uspcontroldevs,2), lw=3,color = 'k', linestyle = ':',label='$2 \sigma$')
plt.axvline(x = np.percentile(uspcontroldevs,98), lw=3,color = 'k', linestyle = ':')
plt.xlabel('Sample Velocity Dispersion [km/s]')
#plt.legend()
#plt.title('{} Iterations'.format(mciters))
plt.ylabel('N')
plt.legend(fontsize=12, frameon=False)
#plt.xlim(35.75,51)
plt.xticks(np.arange(36,52,2))
ax=plt.gca()
ax.yaxis.set_minor_locator(plt.MultipleLocator(25))
ax.xaxis.set_minor_locator(plt.MultipleLocator(1))
#plt.annotate(s = "Warm/Cold Jupiter Hosts:$\ {:.3f}\%$".format(len(controldevs2[controldevs2<hjdev2])*100/len(controldevs2)), xy = (0.5,.95), xycoords = 'axes fraction', fontsize = 14)
#plt.savefig('PaperPlots/wcj_hist.png',bbox_inches='tight')
plt.savefig('PaperPlots/usp_hist.pdf',bbox_inches='tight')

In [ ]:
print(len(uspcontroldevs[uspcontroldevs<uspdev])/len(uspcontroldevs)*100, ' %')

In [ ]:
def mc_control_deviations_samp(hjs, control, sampsize, niters = 1000, metallicity=False, z_distribution=True, delta_M_G0_cut=1.):
    mciters = niters
    hjnongiants = keep_nongiants(hjs,delta_M_G0_cut=delta_M_G0_cut)
    nongiants = keep_nongiants(control,delta_M_G0_cut=delta_M_G0_cut)
    total_deviations = np.ones(mciters)*np.nan
    hj_deviations = np.ones(mciters)*np.nan
    for i in range(mciters):
        subsamp=hjnongiants.sample(sampsize)
        if metallicity:
            control_sample=match_metallicity(subsamp, nongiants)
        if z_distribution:
            control_sample=match_zdistribution(subsamp, nongiants)
        else: 
            control_sample=nongiants
        bprpcontrol = control_sample.bp_rp0
        if i%(mciters/10) == 0:
            print('i = ',i, datetime.datetime.now())
        control_sample_inds = []
        for j, hj in subsamp.iterrows():
            bprprange = 0.025
            bprp = hj.bp_rp0
            bounds = (bprp-bprprange,bprp + bprprange)
            selection=control_sample.loc[np.logical_and(bprpcontrol>bounds[0], bprpcontrol<bounds[1])]
            while len(selection)==0:
                bprprange+=.005
                bounds = (bprp-bprprange,bprp + bprprange)
                selection = control_sample.loc[np.logical_and(bprpcontrol>bounds[0], bprpcontrol<bounds[1])]
            control_sample_inds.append(selection.sample(n=1).index.values)
        control_sample_inds = np.concatenate(control_sample_inds)
        control_sample = nongiants.loc[control_sample_inds]
        total_deviationi = total_deviation(control_sample)
        total_deviations[i] = total_deviationi/len(control_sample[~control_sample.gal_U.isnull()])
        hj_deviations[i] = total_deviation(subsamp)
    #(hjnongiants.Bmag-hjnongiants.Vmag).hist()
    #(control_sample.Bmag-control_sample.Vmag).hist()
    #plt.scatter(hjnongiants.bp_rp0, hjnongiants.M_G0, label = 'Gaia HJs')
    #plt.scatter(control_sample.bp_rp0, control_sample.M_G0, label = 'Final Control Sample')
    #plt.gca().invert_yaxis()
    return hj_deviations, total_deviations


In [ ]:
hj_devs, total_devs = mc_control_deviations_samp(hotjupiters_ms, gaiams, 30, 1000)

In [ ]:
font={'size':16}
plt.rc('font',**font)
fig = plt.figure(figsize=(7.1,7.1*.5))
plt.hist(hj_devs/30, bins = 100,alpha=0.5,label='30 HJs')
#maxy = plt.gca().get_ylim()[1]
plt.hist(total_devs, bins=100,alpha=0.5,label='Control')
plt.axvline(x = np.percentile(total_devs,50), label = 'Median', lw=3,color = 'orange',alpha=0.5)
plt.axvline(x = np.percentile(total_devs,16,), lw=3,color = 'orange', linestyle = '--',label='$1 \sigma$',alpha=0.5)
plt.axvline(x = np.percentile(total_devs,84), lw=3,color = 'orange', linestyle = '--',alpha=0.5)
plt.axvline(x = np.percentile(total_devs,2), lw=3,color = 'orange', linestyle = ':',label='$2 \sigma$',alpha=0.5)
plt.axvline(x = np.percentile(total_devs,98), lw=3,color = 'orange', linestyle = ':',alpha=0.5)
plt.axvline(x = np.percentile(hj_devs/30,50), label = 'Median', lw=3,color = 'blue',alpha=0.5)
plt.axvline(x = np.percentile(hj_devs/30,16,), lw=3,color = 'blue', linestyle = '--',label='$1 \sigma$',alpha=0.5)
plt.axvline(x = np.percentile(hj_devs/30,84), lw=3,color = 'blue', linestyle = '--',alpha=0.5)
plt.axvline(x = np.percentile(hj_devs/30,2), lw=3,color = 'blue', linestyle = ':',label='$2 \sigma$',alpha=0.5)
plt.axvline(x = np.percentile(hj_devs/30,98), lw=3,color = 'blue', linestyle = ':',alpha=0.5)
plt.xlabel('Sample Velocity Dispersion [km/s]')
#plt.legend()
#plt.title('{} Iterations'.format(mciters))
plt.ylabel('N')
plt.legend(fontsize=12, frameon=False)
#plt.xlim(35.75,51)
#plt.xticks(np.arange(36,52,2))
ax=plt.gca()
#ax.yaxis.set_minor_locator(plt.MultipleLocator(25))
#ax.xaxis.set_minor_locator(plt.MultipleLocator(1))
#plt.annotate(s = "Warm/Cold Jupiter Hosts:$\ {:.3f}\%$".format(len(controldevs2[controldevs2<hjdev2])*100/len(controldevs2)), xy = (0.5,.95), xycoords = 'axes fraction', fontsize = 14)
#plt.savefig('PaperPlots/wcj_hist.png',bbox_inches='tight')
#plt.savefig('PaperPlots/usp_hist.pdf',bbox_inches='tight')

In [ ]:
hotjupiters_ms.distance.describe()